## AES128 Encryption and Decryption

In [ ]:
import numpy as np
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'int':hex})
import IPython
from siha import *
import pad
from IPython.display import Image
filename = 'images/andromeda.jpg'
efilename = 'images/andromeda128_enc.jpg'
dfilename = 'images/andromeda128_dec.jpg'

# AES Key Configuration Data
encKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000001, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)

decKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000000, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)

img = image.imread(filename)
img =np.array(img) 
x_axis = img.shape[0]
y_axis = img.shape[1];

In [ ]:
Image(filename=filename)

# Encryption

In [ ]:
slot_AES128 = 0 # change slot number here if required

In [ ]:
BUFFSIZE = 0x1000000
databuffOrg = np.fromfile(filename, dtype=np.uint8)[:BUFFSIZE]
databuff, padcount = pad.addpad(databuffOrg)
databuff = np.frombuffer(databuff.tobytes(), dtype=np.uint32)
size = databuff.shape[0]

tid_config = 1
tid_run = 0
offset_key = 0
offset_input = 64
offset_result = 64
size_config = len(encKeybuff)

accel = AcapdAccel("AES128", slot_AES128)
accel.loadData(encKeybuff, offset_key)
accel.loadData(databuff, offset_input)
accel.startAccel()
accel.config(offset_key, size_config, tid_config)
accel.run(offset_input, size, tid_run, offset_result)
outbuff = accel.retrieve(offset_result, size)
accel.close()
outbuff.tofile(efilename)
ratio = np.sqrt(size/(x_axis*y_axis*3))
x_axis, y_axis = int(x_axis*ratio), int(y_axis*ratio)
img = np.frombuffer(outbuff.tobytes(), dtype=np.uint8)[-x_axis* y_axis* 3:].reshape((x_axis, y_axis, 3))
pyplot.imshow(img)
pyplot.show();

# Decryption

In [ ]:
slot_AES128 = 0 # change slot number here if required

In [ ]:
BUFFSIZE = 0x1000000
databuff = np.fromfile(efilename, dtype=np.uint32)
accel = AcapdAccel("AES128", slot_AES128)
accel.loadData(decKeybuff, offset_key)
accel.loadData(databuff, offset_input)
accel.startAccel()
accel.config(offset_key, size_config, tid_config)
accel.run(offset_input,size, tid_run, offset_result)
outbuff = accel.retrieve(offset_result, size)
accel.close()
outbuff = np.frombuffer(outbuff.tobytes(), dtype=np.uint8)[:-padcount]
outbuff.tofile(dfilename)
img = image.imread(dfilename)
pyplot.imshow(img)
pyplot.show();

In [ ]:
Image(filename=dfilename)